# Gradient Descent

## Linear Regression with L2-Regularization (Ridge Regression)

### Requirements

Libraries needed
- numpy
- scikit-learn (for testing the algorithm)

## Libraries

In [1]:
import numpy as np

## Stochastic Gradient Class

In [2]:
class SGD_Optimizer:
    def __init__(self, batch_size, epochs, epsilon, learning_rate, regularization):
        '''
        Initializes the class with the configuration parameters. Also sets up
        a loss history list and weight attribute.
        
        Parameters
        ----------
        batch_size: int
            the size of batch to iterate over the dataset
        epochs: int
            the maximum number of epochs to run the Gradient Descent Algorithm
        epsilon: float
            the minimum update required to proceed to next epoch
        learning_rate: float
            the learning rate for Gradient Descent
        regularization: float
            the regularization strength for L2-Regularization
        
        Returns
        -------
        None
        '''
        self.epochs = epochs
        self.epsilon = epsilon
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.regularization = regularization
        self.loss = []
        self.W = None
        
    def fit(self, X, Y):
        '''
        Function call to update the weights using Stochastic Gradient Descent
        
        Parameters
        ----------
        X: numpy array or list
            the features on which to learn the weights
        Y: numpy array or list
            the target values
            
        Returns
        -------
        None
        '''
        
        # utility function to calculate Mean Squared Error
        def MSE(y_true, y_pred):
            return np.mean(np.square(y_true - y_pred))
        
        # change to numpy array if list
        if isinstance(X, list):
            X = np.asarray(X)
        if isinstance(Y, list):
            Y = np.asarray(Y)
        
        # if only 1-D array, change to 2-D array
        if len(X.shape)==1:
            X = np.expand_dims(X, -1)
        # append a column of 1 for bias term
        X = np.insert(X, 0, 1, axis=1)
        
        # weights to learn, initialized randomly
        self.W = np.random.randn(X.shape[1])
        
        # indexes for quick shuffling of the X, Y arrays
        indexes = np.asarray([i for i in range(X.shape[0])])
        
        # iterating for epochs
        for _ in range(self.epochs):
            # shuffle the indexes
            np.random.shuffle(indexes)
            
            # track the loss for each batch iteration over X, Y
            epochLoss = []
            
            # batchwise iteration
            for i in range(0, X.shape[0], self.batch_size):
                # extract the batches
                batchX = X[indexes][i:min(i+self.batch_size, X.shape[0])]
                batchY = Y[indexes][i:min(i+self.batch_size, X.shape[0])]
                
                # predictions using the batch of X
                preds = np.matmul(batchX, self.W)
                
                # error associated with this batch
                error = preds - batchY
                
                # append the loss to loss tracking of this epoch
                epochLoss.append(MSE(batchY, preds) + self.regularization*np.sum(np.square(self.W)))
                
                # calculate gradient with L2-regularization
                gradient = (np.matmul(batchX.T, error)/batchX.shape[0]) + self.regularization*self.W
                
                # update weights
                self.W += -1*self.learning_rate*gradient
            
            # append mean loss of this epoch to loss history
            self.loss.append(np.mean(epochLoss))
            
            # break if the last update was not significant
            if len(self.loss) > 1 and abs(self.loss[-1] - self.loss[-2]) < self.epsilon:
                break
                
    def predict(self, X):
        '''
        Function to predict the target values using features provided
        
        Parameters
        ----------
        X: numpy array or list
            the features on which to predict the target values
            
        Returns
        -------
        numpy.ndarray
            returns the predicted values
        '''
        # change to numpy array if list
        if isinstance(X, list):
            X = np.asarray(X)
        
        # if only 1-D array, change to 2-D array
        if len(X.shape)==1:
            X = np.expand_dims(X, -1)
        
        # append a column of 1 for bias term
        X = np.insert(X, 0, 1, axis=1)
        return np.matmul(X, self.W)

## Execution Example using Boston Housing Dataset

### Libraries

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston

### Load the Boston Housing Dataset

In [4]:
X, Y = load_boston(return_X_y=True)

### Split the data into train and test split

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

### Initialize the Stochastic Gradient Descent Optimizer

In [6]:
optimizer = SGD_Optimizer(batch_size=10, epochs=1000, epsilon=0.01, learning_rate=0.001, regularization=0.01)

### Normalize the dataset using train mean and standard deviation

In [7]:
train_min = X_train.min(0)
train_max = X_train.max(0)
X_train = (X_train - train_min)/(train_max - train_min)
X_test = (X_test - train_min)/(train_max - train_min)

### Fit the optimizer on training data

In [8]:
optimizer.fit(X_train, Y_train)

### Training Loss Progression

In [9]:
print('Epoch\tLoss')
for i, loss in enumerate(optimizer.loss):
    print('{}:\t{:.4f}'.format(i, loss))

Epoch	Loss
0:	485.7087
1:	399.1401
2:	340.7033
3:	283.8458
4:	240.5925
5:	213.5142
6:	187.2385
7:	170.3185
8:	158.6280
9:	147.0011
10:	135.8496
11:	134.5673
12:	122.1231
13:	119.3382
14:	118.8978
15:	113.9550
16:	107.6603
17:	108.1371
18:	107.0517
19:	103.4906
20:	101.2914
21:	102.5263
22:	98.5510
23:	97.4385
24:	96.1608
25:	95.7884
26:	99.2636
27:	94.0721
28:	96.2951
29:	91.3991
30:	90.7666
31:	90.1548
32:	91.8037
33:	89.0630
34:	89.7787
35:	87.2864
36:	86.6605
37:	86.8719
38:	87.6939
39:	84.5699
40:	83.9814
41:	85.0006
42:	82.9549
43:	84.8624
44:	81.5445
45:	80.7272
46:	80.8607
47:	83.4091
48:	85.9534
49:	79.2474
50:	79.3939
51:	77.7387
52:	78.1295
53:	76.8117
54:	76.9142
55:	75.7832
56:	75.5087
57:	74.9654
58:	78.4401
59:	76.2479
60:	73.5005
61:	73.6442
62:	73.2269
63:	72.6652
64:	72.4734
65:	72.7348
66:	72.0700
67:	71.6179
68:	74.0642
69:	73.0508
70:	70.8317
71:	70.0466
72:	72.9540
73:	69.5749
74:	69.5369
75:	71.0103
76:	68.0934
77:	69.0146
78:	70.0411
79:	67.3681
80:	67.0520
81:	6

### Predict the target values for test set

In [10]:
Y_pred = optimizer.predict(X_test)

In [11]:
print('Sample\tTruth\tPredicted')
for i in range(len(Y_test)):
    print('{}:\t{:.1f}\t{:.3f}'.format(i, Y_test[i], Y_pred[i]))

Sample	Truth	Predicted
0:	23.6	24.779
1:	32.4	29.238
2:	13.6	21.967
3:	22.8	23.176
4:	16.1	19.448
5:	20.0	23.648
6:	17.8	23.215
7:	14.0	22.113
8:	19.6	20.577
9:	16.8	22.983
10:	21.5	26.168
11:	18.9	24.201
12:	7.0	6.978
13:	21.2	23.433
14:	18.5	23.675
15:	29.8	20.396
16:	18.8	23.103
17:	10.2	15.357
18:	50.0	29.486
19:	14.1	19.580
20:	25.2	24.362
21:	29.1	25.412
22:	12.7	22.304
23:	22.4	25.079
24:	14.2	17.582
25:	13.8	17.530
26:	20.3	22.797
27:	14.9	11.122
28:	21.7	26.845
29:	18.3	22.458
30:	23.1	22.958
31:	23.8	24.732
32:	15.0	21.563
33:	20.8	18.654
34:	19.1	17.632
35:	19.4	18.569
36:	34.7	27.055
37:	19.5	23.744
38:	24.4	26.094
39:	23.4	22.858
40:	19.7	21.068
41:	28.2	26.424
42:	50.0	30.008
43:	17.4	23.265
44:	22.6	24.922
45:	15.1	18.807
46:	13.1	22.646
47:	24.2	23.392
48:	19.9	19.812
49:	24.0	26.300
50:	18.9	24.696
51:	35.4	26.066
52:	15.2	24.143
53:	26.5	25.155
54:	43.5	26.338
55:	21.2	19.747
56:	18.4	19.755
57:	28.5	27.930
58:	23.9	24.778
59:	18.5	23.746
60:	25.0	25.982
61:	35.4	29.2